In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load images
rgb_image = cv2.imread('IMG_9395.JPG') 
depth_image = cv2.imread('path_to_output_image.jpg', cv2.IMREAD_UNCHANGED) 
depth = cv2.resize(depth_image, (550, 336), interpolation=cv2.INTER_CUBIC)
rgb_image = cv2.resize(rgb_image, (550, 336), interpolation=cv2.INTER_CUBIC)

# Create depth mask
mask_depth = (depth > 12).astype(np.uint8)
structuring_element = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
dilated_array = cv2.dilate(mask_depth, structuring_element)
mask_depth = cv2.morphologyEx(dilated_array, cv2.MORPH_CLOSE, structuring_element)


# Blur and edge detection
blurred = cv2.GaussianBlur(rgb_image[:, :, 2], (5, 5), 0)
edges = cv2.Canny(blurred, threshold1=100, threshold2=150)
masked_edges = cv2.bitwise_and(edges, mask_depth)

# Close to make a continous edge
masked_edges = cv2.morphologyEx(masked_edges, cv2.MORPH_CLOSE, np.ones((5, 5)))

# Find contours and filter to remove small contours
contours, _ = cv2.findContours(masked_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
min_area_threshold = 200 
filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area_threshold]

# Bounding Boxes
image_with_bounding_boxes = rgb_image.copy()
for cnt in filtered_contours:
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(image_with_bounding_boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)

#Show extracted region
mask = np.zeros_like(masked_edges)
cv2.drawContours(mask, filtered_contours, -1, 255, -1)
mask_3channel = np.repeat(mask[:, :, np.newaxis], 3, axis=2)

new_image = cv2.bitwise_and(rgb_image[:, :, 2], mask)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Bounding Boxes on RGB Image')
plt.imshow(cv2.cvtColor(image_with_bounding_boxes, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.subplot(1, 2, 2)
plt.title('Extracted Region')
plt.imshow(new_image, cmap='gray')
plt.axis('off')

plt.show()
